In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/foodrec/category_data_test.csv')
data

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department,category
0,1,49302,1,1,Bulgarian Yogurt,120,16,112108,train,4,4,10,9,yogurt,dairy eggs,yogurt
1,816049,49302,7,1,Bulgarian Yogurt,120,16,47901,train,14,4,6,16,yogurt,dairy eggs,yogurt
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,2993,train,15,0,7,7,yogurt,dairy eggs,yogurt
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,41425,train,4,3,8,14,yogurt,dairy eggs,yogurt
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,187205,train,5,4,14,30,yogurt,dairy eggs,yogurt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447068,1713171,36616,1,1,Gin,124,5,6371,train,6,3,16,1,spirits,alcohol,alcohol
447069,1552485,47031,2,1,Kentucky Straight Bourbon Whiskey,124,5,205384,train,10,0,10,15,spirits,alcohol,alcohol
447070,1888595,47031,1,1,Kentucky Straight Bourbon Whiskey,124,5,86949,train,5,6,18,6,spirits,alcohol,alcohol
447071,2773302,234,1,0,Tennessee Whiskey,124,5,177868,train,4,6,14,30,spirits,alcohol,alcohol


## 유저가 함께 샀던 제품들로만 df 구성 -> 유사도 측정 -> 상품 추천

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import itertools

def recommend_product(p):
    case1_data = data.drop(columns=['add_to_cart_order', 'reordered','aisle_id','department_id','product_name','order_id', 'department',
                                'category', 'eval_set','order_number','order_dow','order_hour_of_day','days_since_prior_order'], axis=1)

    # 상품과 함께 구매했던 다른 상품 list로 만들기
    user_list = list(data[data['product_id'] == p]['user_id'])
    product_pay_list = []
    for i in user_list:
        product_pay_list.append(list((case1_data[case1_data['user_id'] == i]['product_id'])))
        product_list = list(itertools.chain.from_iterable(product_pay_list))

    if len(product_pay_list) != 0: # 기존에 있는 제품일 경우에는    
        df_list = sorted(set(product_list)) # 상품과 함께 구매했던 다른 상품 df형식으로 만들고
        df = pd.DataFrame(columns = ['product_id', 'user_id', 'aisle'])
        for i in range(len(df_list)):
            df = df.append(pd.DataFrame(case1_data[case1_data['product_id'] == df_list[i]]))

        # 상품간 유사도 측정
        user_matrix_product = pd.crosstab(df.user_id, df.product_id)
        user_matrix_product_t = user_matrix_product.transpose()
        sim_data=cosine_similarity(user_matrix_product_t)
        index = np.where(user_matrix_product_t.index == p)[0][0]
        similar_aisle = sorted(list(enumerate(sim_data[index])),key = lambda x:x[1],reverse = True)[1:10]
        
        # best-seller 모델 구현
        def rec_product(n_items): # 상위 30개중에 랜덤 9개 추출
            product_count = data['product_id'].value_counts().to_frame()
            product_count = product_count.reset_index().rename(columns = {"index":"product_id","product_id":"count","count":"category"})
            change_data = pd.merge(data, product_count, on = "product_id")
            df = change_data.drop(columns=['add_to_cart_order','department_id','order_id','user_id','eval_set','reordered','product_name',
                                    'aisle_id','category','order_number','order_dow','order_hour_of_day','days_since_prior_order','aisle','department'], axis=1)
            df = df.drop_duplicates()
            final_df = df.sort_values('count', ascending = False).head(30)
            data_top30 = list(set(final_df['product_id']))
            top_sample = random.sample(data_top30, k=9)
            return top_sample

        # 최종 상품 추천    
        brr = []
        for i in similar_aisle:
            brr.append(user_matrix_product_t.index[i[0]])
        # 만약 추천 갯수가 9개 이하일 경우는 BEST-SELLER 모델에서 추천하도록 코드 구현
        if len(brr) <9:
            brr.append(int(''.join([(str(n)) for n in rec_product(9-len(brr))])))
        return brr    

    else: # 새로운 제품이 입력될 경우에           
        def rec_long_tail(n_items): # long-tail 제품 추천
            product_count = data['product_id'].value_counts().to_frame()
            product_count = product_count.reset_index().rename(columns = {"index":"product_id","product_id":"count","count":"category"})
            change_data = pd.merge(data, product_count, on = "product_id")
            df = change_data.drop(columns=['add_to_cart_order','department_id','order_id','user_id','eval_set','reordered','product_name',
                                    'aisle_id','category','order_number','order_dow','order_hour_of_day','days_since_prior_order','aisle','department'], axis=1)
            df = df.drop_duplicates()
            final_df = df.sort_values('count', ascending = True).head(30)
            data_long30 = list(set(final_df['product_id']))
            tail_sample = random.sample(data_long30, k=n_items)
            return tail_sample
        return rec_long_tail(9)  

recommend_product(21560)

[7330, 5214, 32412, 4116, 6462, 14100, 27651, 30699, 33989]

## 다른 방식) aisle간 유사도 측정 -> 구매한 상품끼리 구성 -> aisle에 속하지않은 애들 삭제 -> 추천

In [7]:
import random
import itertools
from sklearn.metrics.pairwise import cosine_similarity

def recommend_product(p):
    case1_data = data.drop(columns=['add_to_cart_order', 'reordered','aisle_id','department_id','product_name','order_id', 'department',
                                'category', 'eval_set','order_number','order_dow','order_hour_of_day','days_since_prior_order'], axis=1)
    aisle = case1_data[case1_data['product_id'] == p]['aisle'].values[0]

    # 중분류 추천시스템
    user_matrix_aisle = pd.crosstab(case1_data.user_id, case1_data.aisle) # 중분류 유저간 행렬 구성
    user_matrix_aisle_t = user_matrix_aisle.transpose() # 전치행렬 구성
    sim_data=cosine_similarity(user_matrix_aisle_t) # 코사인 유사도 측정
    index = np.where(user_matrix_aisle_t.index == aisle)[0][0]
    similar_aisle = sorted(list(enumerate(sim_data[index])),key = lambda x:x[1],reverse = True)[:3]
    
    arr = []
    for i in similar_aisle:
        arr.append(user_matrix_aisle_t.index[i[0]])

    user_list = list(case1_data[case1_data['product_id'] == p]['user_id'])
    product_pay_list = []
    for i in user_list:
        product_pay_list.append(list((data[data['user_id'] == i]['product_id'])))
        product_list = list(itertools.chain.from_iterable(product_pay_list))

    # 상품과 함께 구매했던 다른 상품 df형식으로 만들고
    df_list = sorted(set(product_list))
    df = pd.DataFrame(columns = ['product_id', 'user_id', 'aisle', 'department'])
    for i in range(len(df_list)):
        df = df.append(pd.DataFrame(data[data['product_id'] == df_list[i]]))

    # 상품 중에 중분류에 속하는 애들만 df상에서 골라내고
    df_c = df[(df['aisle'] == arr[0])|(df['aisle'] == arr[1])|(df['aisle'] == arr[2])]
    df_final = pd.DataFrame(df_c)

    # 상품간 유사도 측정
    user_matrix_product = pd.crosstab(df_final.user_id, df_final.product_id)
    user_matrix_product_t = user_matrix_product.transpose()
    sim_data=cosine_similarity(user_matrix_product_t)
    index = np.where(user_matrix_product_t.index == p)[0][0]
    similar_aisle = sorted(list(enumerate(sim_data[index])),key = lambda x:x[1],reverse = True)[1:10]
    
    # best-seller 모델 구현
    def rec_product(n_items):
        product_count = data['product_id'].value_counts().to_frame()
        product_count = product_count.reset_index().rename(columns = {"index":"product_id","product_id":"count","count":"category"})
        change_data = pd.merge(data, product_count, on = "product_id")
        df = change_data.drop(columns=['add_to_cart_order','department_id','order_id','user_id','eval_set',
                               'order_number','order_dow','order_hour_of_day','days_since_prior_order','aisle','department'], axis=1)
        df = df.drop_duplicates()
        final_df = df.sort_values('count', ascending = False)
        best_reordered_model = final_df[final_df['reordered'] == 1]

        data_top30 = best_reordered_model['product_id']
        top30_product = data_top30.value_counts().head(30)
        top30_product_list = list(top30_product.index)
        top_sample = random.sample(top30_product_list, k=n_items)
        return top_sample

    # 최종 상품 추천    
    brr = []
    for i in similar_aisle:
        brr.append(user_matrix_product_t.index[i[0]])
    # 만약 추천 갯수가 9개 이하일 경우는 BEST-SELLER 모델에서 추천하도록 코드 구현
    if len(brr) <9:
        brr.append(int(''.join([(str(n)) for n in rec_product(9-len(brr))])))
    return brr    

recommend_product(47031)

[13895, 5146, 38347, 16275, 234, 38569, 6137, 27652, 37923]

구현하는 데에 비교적 시간이 오래걸려서 이는 사용하지 않기로 함